In [1]:
import logging
import uuid
from pathlib import Path

import cv2
import depthai as dai # access the camera and its data packets
#from imutils.video import FPS

In [2]:
log = logging.getLogger(__name__)

In [3]:
# Start defining a pipeline
pipeline = dai.Pipeline()

In [4]:
# Define a source - color camera
cam_rgb = pipeline.createColorCamera()
cam_rgb.setPreviewSize(300, 300)
cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam_rgb.setInterleaved(False)
cam_rgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# Create output
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb_preview_out")
cam_rgb.preview.link(xout_rgb.input)

In [5]:
# Define a source - two mono (grayscale) cameras
left = pipeline.createMonoCamera()
left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
left.setBoardSocket(dai.CameraBoardSocket.LEFT)

right = pipeline.createMonoCamera()
right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
right.setBoardSocket(dai.CameraBoardSocket.RIGHT)

# Create a node that will produce the depth map (using disparity output as it's easier to visualize depth this way)
depth = pipeline.createStereoDepth()
depth.setConfidenceThreshold(200) # TODO:  what are the different values for this threshold?
left.out.link(depth.left)
right.out.link(depth.right)

# Create output
xout = pipeline.createXLinkOut()
xout.setStreamName("disparity")
depth.disparity.link(xout.input)

In [6]:
output = pipeline.getAllAssets()

In [7]:
# Pipeline defined, now the device is connected to
with dai.Device(pipeline) as device:
    # Start pipeline
    device.startPipeline()

    # Output queues will be used to get the grayscale / depth frames and nn data from the outputs defined above
    rgb = device.getOutputQueue("rgb_preview_out", maxSize=4, blocking=False)
    qDisparity = device.getOutputQueue("disparity", maxSize=4, blocking=False)

    rgbFrame = None
    disparityFrame = None

    while True:
        inRight = rgb.get()
        inDisparity = qDisparity.get()

        rightFrame = inRight.getFrame()q
        rightFrame = cv2.flip(rightFrame, flipCode=1)
        #cv2.imshow("rectified right", rightFrame)


        disparityFrame = inDisparity.getFrame()
        cv2.imshow("disparity", disparityFrame)

        if cv2.waitKey(1) == ord('q'):
            cv2.destroyAllWindows()  # needed to avoid openCV from crashing
            break